# DEEP NEURAL NETWORKS CLASSIFICATION 2

классификация тематик обращений клиентов к чат-боту с помощью модели на основе двунаправленной рекуррентной сети LSTM и Dense.

Данильченко Вадим

-------------------------------------------------------------------------------------------------------------------------------

class	intent
- 0	узнать возраст бота
- 1	узнать какую музыку слушает бот
- 2	узнать любит ли бот путешествовать
- 3	узнать где работает бот
- 4	узнать информацию про семью бота
- 5	узнать как относится бот к спорту
- 6	узнать где учится бот
- 7	узнать что читает бот
- 8	узнать есть ли у бота животные
- 9	узнать есть ли у бота машина
- 10	узнать как выглядит бот
- 11	узнать как зовут бота
- 12	узнать какие парни/девушки нравятся
- 13	узнать какой пол у бота
- 14	узнать кто такой бот
- 15	узнать кулинарные пристрастия бота
- 16	узнать о чем мечтает бот
- 17	поблагодарить бота
- 18	поприветствовать бота
- 19	попрощаться с ботом
- 20	разослать информацию студентам на email
- 21	узнать как дела
- 22	узнать метеопрогноз на сегодня
- 23	узнать об увлечениях
- 24	узнать откуда бот
- 25	узнать расписание занятий
- 26	узнать чем занимается бот
- 27	услышать анекдот
- 28	узнать о преподавателе
- 29	сделать уведомление студентам канала
- 30	узнать что умеет бот

In [3]:
from tensorflow import keras

In [4]:
from keras import Sequential
from keras.layers import Dense, Flatten, Conv1D, Dropout, Embedding, MaxPooling1D, Conv2D, LSTM, Bidirectional

Using TensorFlow backend.


In [5]:
import joblib
import numpy as np

##### DATA

In [6]:
with open(r'F:\ОБУЧЕНИЕ\CHAT-BOTs\тексты\dialogues\_phrases2.txt', encoding='utf-8') as f:
    phrases = f.readlines()
phrases = [i.replace('\n', '') for i in phrases]
with open(r'F:\ОБУЧЕНИЕ\CHAT-BOTs\тексты\dialogues\_intents2.txt', encoding='utf-8') as f:
    intents = f.readlines()    
intents = [i.replace('\n', '') for i in intents]

In [8]:
from gensim.models import Word2Vec
from keras.preprocessing.text import text_to_word_sequence

def preproc(phrase):
    return text_to_word_sequence(phrase, 
                              filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                              lower=True,
                              split=' ')

text = [preproc(phrase) for phrase in phrases]

In [ ]:
wvec = Word2Vec(text, size=100, window=5, min_count=3, workers=4, )

In [6]:
def vectorize(phrase):
    temp_vector=[]
    if isinstance(phrase, list)==False:
        phrase=preproc(phrase)
    for word in phrase:
        if word not in wvec.wv:
            temp_vector.append([0.0]*100)
        else:
            temp_vector.append(wvec.wv.__getitem__(word))
    result = np.mean(temp_vector, axis=0)
    if result.all():
        return result
    else:
        return np.array([0.0]*100)
#     return [0.0]*100 if np.mean(temp_vector, axis=0)==None else 1

X = [vectorize(phrase) for phrase in text]
len(X)

539

In [7]:
X[0]

array([ 2.4606751e-03, -3.2825675e-04,  5.7301082e-04,  3.6444948e-03,
        2.6223657e-03,  1.1212626e-03,  4.9434537e-03, -3.8440598e-03,
        8.2128181e-04, -9.1157458e-04,  2.8706954e-03, -3.8842673e-03,
        6.9822290e-04,  8.9537696e-04, -3.1997052e-03, -3.3748092e-03,
       -2.4595617e-03, -4.6567689e-03,  1.2049159e-03,  2.2083134e-03,
       -4.8366082e-03,  4.4095428e-03, -2.8277508e-03, -3.8247774e-03,
       -9.5037592e-04,  1.0915499e-03,  7.7794550e-04, -5.8420561e-04,
       -3.5331023e-03, -3.5383569e-03, -2.8759532e-03,  1.9167886e-05,
       -1.2246346e-04,  4.0792092e-03, -2.5710443e-03,  2.0383559e-03,
        4.2876485e-03,  7.5289317e-05, -1.4859586e-03,  3.0552552e-03,
       -4.8747538e-03, -4.0660487e-03,  1.3579447e-03,  1.9121925e-04,
        3.9888653e-03,  2.0053533e-03,  1.3379848e-03,  4.6574064e-03,
        1.0843552e-03,  3.9194147e-03,  3.0413237e-03, -3.2763164e-03,
        7.4192358e-04, -2.1184881e-03,  3.2912779e-03, -3.4556394e-03,
      

In [8]:
X[0].shape

(100,)

##### positions of words

In [292]:
# объединим в один список все предложения
full_list = []
for phrase in text:
    full_list += phrase
# составим список уникальных слов
words_list = list(set(full_list))
# создадим словарь слово в номер
word2index = {v:k for k,v in enumerate(words_list)}
# создадим обратный словарь
index2word = {v:k for k,v in word2index.items()}
# создадим список текстов, где вместо слова его индекс
def text_to_word_indexes(phrase):
    temp_list=[]
    for word in phrase:
        temp_list.append(word2index[word])
    return temp_list

text_indexes = []
for phrase in text:
    text_indexes.append(np.asarray(text_to_word_indexes(phrase)))

In [293]:
text_indexes

[array([18]),
 array([ 2, 25]),
 array([50]),
 array([23]),
 array([55]),
 array([35]),
 array([48]),
 array([ 2, 72]),
 array([53, 38]),
 array([17, 49]),
 array([82]),
 array([58, 32]),
 array([58]),
 array([29]),
 array([9]),
 array([9, 9]),
 array([ 6, 56, 62]),
 array([ 6, 46]),
 array([45, 67]),
 array([45, 44, 67]),
 array([45, 75]),
 array([45, 77]),
 array([45, 41]),
 array([85, 45]),
 array([45, 73]),
 array([47, 27]),
 array([12, 45, 26]),
 array([14, 37]),
 array([79, 21, 26, 39]),
 array([27, 13]),
 array([64, 14]),
 array([30, 83]),
 array([14, 70, 26, 31]),
 array([42, 24, 28]),
 array([61, 28, 36, 51]),
 array([65, 16, 28]),
 array([43, 22, 68,  8]),
 array([69, 20]),
 array([21, 33, 78]),
 array([74, 57, 54,  1]),
 array([54, 52]),
 array([40,  4]),
 array([ 7, 60, 11]),
 array([84, 74, 80, 63]),
 array([71,  5]),
 array([76, 63]),
 array([84, 15, 10]),
 array([63, 74, 59, 19,  3, 59, 66]),
 array([ 0, 34, 81, 15]),
 array([18]),
 array([ 2, 25]),
 array([50]),
 array(

In [81]:
from keras.preprocessing.sequence import pad_sequences

In [93]:
max_len_sequnce=0
for phrase in text_indexes:
    if len(phrase)>max_len_sequnce:
        max_len_sequnce=len(phrase)
max_len_sequnce

7

In [100]:
pad_sequences([text_indexes[1]], maxlen=max_len_sequnce, padding='post')

array([[18,  0,  0,  0,  0,  0,  0]])

In [277]:
pad_seq(text_indexes[0])

In [287]:
text

[['привет'],
 ['добрый', 'день'],
 ['приветик'],
 ['здарова'],
 ['хай'],
 ['hi'],
 ['hello'],
 ['добрый', 'вечер'],
 ['доброй', 'ночи'],
 ['доброе', 'утро'],
 ['пока'],
 ['счастливо', 'оставаться'],
 ['счастливо'],
 ['адьес'],
 ['bye'],
 ['bye', 'bye'],
 ['до', 'новых', 'встреч'],
 ['до', 'встречи'],
 ['как', 'дела'],
 ['как', 'твои', 'дела'],
 ['как', 'делишки'],
 ['как', 'оно'],
 ['как', 'жизнь'],
 ['че', 'как'],
 ['как', 'сам'],
 ['расскажи', 'анекдот'],
 ['пошути', 'как', 'нибудь'],
 ['знаешь', 'приколы'],
 ['напиши', 'что', 'нибудь', 'смешное'],
 ['анекдот', 'хочу'],
 ['шутки', 'знаешь'],
 ['шутить', 'умеешь'],
 ['знаешь', 'какой', 'нибудь', 'прикол'],
 ['сделай', 'рассылку', 'студентам'],
 ['пошли', 'студентам', 'такой', 'текст'],
 ['отправь', 'информацию', 'студентам'],
 ['разошли', 'письма', 'с', 'новостью'],
 ['какая', 'погода'],
 ['что', 'за', 'окном'],
 ['будет', 'ли', 'сегодня', 'дождь'],
 ['сегодня', 'солнечно'],
 ['сколько', 'градусов'],
 ['температуру', 'можешь', 'сказат

In [294]:
text_to_word_indexes(['привет'])

[18]

In [295]:
def pad_seq(phrase):
    return pad_sequences([text_to_word_indexes(phrase)], maxlen=max_len_sequnce, padding='post')[0]

X = []
for phrase in text:
    X.append(pad_seq(phrase))
X=np.asarray(X)
X[:2]

array([[18,  0,  0,  0,  0,  0,  0],
       [ 2, 25,  0,  0,  0,  0,  0]])

In [248]:
X.shape

(539, 7)

##### labels

In [10]:
classes = list(set(intents))
y = [classes.index(i) for i in intents]

In [11]:
y[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [12]:
len(y)

539

##### split to train/test samples

In [13]:
from keras.utils import to_categorical
y_binary = to_categorical(y)

In [14]:
y_binary

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [249]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=444)

In [250]:
len(x_train)

431

##### MODEL

In [260]:
model = Sequential([
    Embedding(len(words_list), 7, input_length=max_len_sequnce),
    Bidirectional(LSTM(32, batch_input_shape=(None, 1, max_len_sequnce), return_sequences=True)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(7, activation='sigmoid')
])

In [261]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [262]:
model.summary()

Model: "sequential_53"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_54 (Embedding)     (None, 7, 7)              602       
_________________________________________________________________
bidirectional_26 (Bidirectio (None, 7, 64)             10240     
_________________________________________________________________
flatten_19 (Flatten)         (None, 448)               0         
_________________________________________________________________
dense_103 (Dense)            (None, 64)                28736     
_________________________________________________________________
dense_104 (Dense)            (None, 7)                 455       
Total params: 40,033
Trainable params: 40,033
Non-trainable params: 0
_________________________________________________________________


In [263]:
model.fit(x_train
          ,np.array(y_train)
          ,batch_size=10
          ,epochs=30
          ,verbose=1 
#           ,validation_split=0.2 
          ,validation_data=(x_test, np.array(y_test))
          ,shuffle=True
         )

Train on 431 samples, validate on 108 samples
Epoch 1/30
431/431 [==============================] - 2s 4ms/step - loss: 1.9443 - accuracy: 0.1810 - val_loss: 1.9422 - val_accuracy: 0.1944
Epoch 2/30
431/431 [==============================] - 1s 2ms/step - loss: 1.9410 - accuracy: 0.2320 - val_loss: 1.9386 - val_accuracy: 0.3333
Epoch 3/30
431/431 [==============================] - 1s 2ms/step - loss: 1.9373 - accuracy: 0.2367 - val_loss: 1.9331 - val_accuracy: 0.2778
Epoch 4/30
431/431 [==============================] - 1s 2ms/step - loss: 1.9316 - accuracy: 0.2367 - val_loss: 1.9254 - val_accuracy: 0.2778
Epoch 5/30
431/431 [==============================] - 1s 2ms/step - loss: 1.9225 - accuracy: 0.2575 - val_loss: 1.9133 - val_accuracy: 0.2963
Epoch 6/30
431/431 [==============================] - 1s 2ms/step - loss: 1.9093 - accuracy: 0.2506 - val_loss: 1.8959 - val_accuracy: 0.2963
Epoch 7/30
431/431 [==============================] - 1s 2ms/step - loss: 1.8917 - accuracy: 0.2575 - 

In [264]:
model.evaluate(np.array(x_train),
              y_train,
              batch_size=10, 
              verbose=1)

431/431 [==============================] - 0s 1ms/step


[0.04068337247346089, 1.0]

In [311]:
model.predict(np.array(pad_seq(['погода'])).reshape(1,-1))

array([[1.4156103e-05, 1.4603138e-06, 1.6093254e-06, 0.0000000e+00,
        2.9182875e-08, 6.6468994e-05, 1.1595703e-05]], dtype=float32)

In [318]:
classes[np.argmax(model.predict(np.array(pad_seq('какая погода'.split())).reshape(1,-1)))]

'узнать метеопрогноз на сегодня'